In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the data
data = pd.read_csv('your_dataset.csv')

# Separate labeled and unlabeled data
labeled_data = data.dropna(subset=['target'])
unlabeled_data = data[data['target'].isnull()].drop('target', axis=1)

# Split labeled data into features and labels
X_labeled = labeled_data.drop('target', axis=1)
y_labeled = labeled_data['target']

# Split labeled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42)

# Create a base classifier (e.g., Decision Tree)
base_classifier = DecisionTreeClassifier()

# Create a SelfTrainingClassifier
self_training_classifier = SelfTrainingClassifier(base_classifier)

# Initial training on labeled data
self_training_classifier.fit(X_train, y_train)

# Pseudo-labeling and iterative training
for iteration in range(num_iterations):
    # Pseudo-labeling on unlabeled data
    pseudo_labels = self_training_classifier.predict(unlabeled_data)

    # Combine labeled and pseudo-labeled data
    X_combined = pd.concat([X_train, unlabeled_data], axis=0)
    y_combined = pd.concat([y_train, pd.Series(pseudo_labels)], axis=0)

    # Train the self-training classifier on the combined dataset
    self_training_classifier.fit(X_combined, y_combined)
    

# Evaluate the final model on your validation set
y_pred = self_training_classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy on validation set: {accuracy:.2f}')


## Import Dependencies

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Load Data

In [2]:
# Load the data
df = pd.read_csv('data\Bengali_hate_speech.csv')
df.head()

sentence  hate category
0                     ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА!!!!!   1.0   sports
1                  ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржирзЗржУрзЯрж╛ ржжрж░ржХрж╛рж░   1.0   sports
2  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ рж╣ржмрзЗ ржПржЯрж╛...   1.0   sports
3                рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржжрзЗржЦрждрзЗ ржкрж╛ржарж╛рж░ ржоржд ржжрзЗржЦрж╛ ржпрж╛рзЯ   1.0   sports
4   рждрзБржЗ рждрзЛ рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм   1.0   sports

In [3]:
df['sentence'][52]

'ржорж╛ржжрж╛рж░ржЪрзБржж <br />рж╢рж╛рж▓рж╛ ржнрж┐ржХрзНрж╖рж╛рж░рзАрж░ ржмрж╛ржЪрзНржЪрж╛<a href="http://www.youtube.com/results?search_query=%23admin">#admin</a>'

## Clean

In [ ]:
def clean(text):
    # remove '<a href="http://www.youtube.com/results?search_query=%23admin">#admin</a>'      
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'#admin', '', text)
    # remove 'ред' (Dari)
    text = re.sub('ред+', ' ', text)
    # remove '< br / >,,,'
    text = re.sub(r'<\s*br\s*/?>,*,?', '', text)
    
    return text

## Remove HTML tag

In [12]:
import re

def remove_html_and_specific_text(text):
    # Define the regular expression pattern for detecting HTML tags
    html_tag_pattern = re.compile(r'<.*?>')
    
    # Use re.sub to replace HTML tags with an empty string
    text_without_html = re.sub(html_tag_pattern, '', text)
    
    # Remove specific text like '#admin'
    text_without_specific_text = re.sub(r'#admin', '', text_without_html)
    
    return text_without_specific_text

In [15]:
df['sentence'] = df['sentence'].apply(remove_html_and_specific_text)

In [16]:
df['sentence'][52]

'ржорж╛ржжрж╛рж░ржЪрзБржж рж╢рж╛рж▓рж╛ ржнрж┐ржХрзНрж╖рж╛рж░рзАрж░ ржмрж╛ржЪрзНржЪрж╛'

## Remove Dari

In [47]:
def remove_dari(text):
    return re.sub('ред+', ' ', text)

In [48]:
df['sentence'] = df['sentence'].apply(remove_dari)

In [50]:
remove_dari('ржХрзБржкрж┐рзЯрзЗ ржПрждрзЛ ржЧрзБрж▓рж╛ ржжрж╛рзЬрж┐рзЯрзЗ ржжрзЗржЦржЫрзЗредредржнрж┐ржбрж┐ржУ ржХрж░ржЫрзЗредредредредрж╣рж╛ржд')

'ржХрзБржкрж┐рзЯрзЗ ржПрждрзЛ ржЧрзБрж▓рж╛ ржжрж╛рзЬрж┐рзЯрзЗ ржжрзЗржЦржЫрзЗ ржнрж┐ржбрж┐ржУ ржХрж░ржЫрзЗ рж╣рж╛ржд'

## Check shape, Null value and Value_Count

In [258]:
df.shape

(19999, 3)

In [259]:
df.isnull().sum()

sentence        0
hate        10002
category        0
dtype: int64

In [260]:
df['hate'].value_counts()

1.0    4999
0.0    4998
Name: hate, dtype: int64

## Preprocessing

In [17]:
import spacy

In [18]:
nlp = spacy.blank("bn")

In [45]:
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.text)
    
    return " ".join(filtered_tokens) 

In [264]:
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'ржХрж╛рж░ржУ', 'рждрж┐ржиржЗ', 'ржирж╛ржирж╛', 'ржерж╛ржХржмрзЗржи', 'ржмрж╛ржжрзЗ', 'ржмржи', 'рж╣ржпрж╝рзЗржЗ', 'рждрзБрж▓рзЗ', 'рж╕рзЗржЯрж┐', 'ржХрзЛржи', 'рж╣рж╛ржЬрж╛рж░', 'ржХрж╛ржЬрзЗ', 'ржорзЛржжрзЗрж░', 'ржХрзЯрзЗржХржЯрж┐', 'ржирж┐ржЬрзЗрж░', 'рждрж╛рж╣рж╛рждрзЗ', 'ржмрж▓рж▓', 'ржЬрж╛ржирж╛ржпрж╝', 'ржЖржжрзНржпржнрж╛ржЧрзЗ', 'ржжрж┐рждрзЗ', 'ржжрзЗржУрзЯрж╛', 'рждрж╛ржжрзЗрж░', 'ржпрж┐ржирж┐', 'ржЫрж╛рзЬрж╛', 'ржжрж┐ржпрж╝рзЗ', 'рж╣рж▓рзЗржЗ', 'рждрж╛рж╣рж╛рж░', 'ржПржБржжрзЗрж░', 'ржирж╛', 'ржПржХржмрж╛рж░', 'ржХрж╛ржЬ', 'рж╣ржпрж╝рзЗржЫрж┐рж▓', 'рж╕ржорж╕рзНржд', 'ржХрзЛржиржУ', 'рж╕рзБрждрж░рж╛ржВ', 'ржХрж░рж╛рзЯ', 'ржлрзЗрж░', 'рж╣рзЯ', 'ржЗрж╣рж╛', 'ржХрж░рж┐ржпрж╝рзЗ', 'рж╣ржд', 'рж╕ржм', 'ржХрзНрж╖рзЗрждрзНрж░рзЗ', 'ржЬржи', 'ржпрж╛рждрзЗ', 'ржорждрзЛржЗ', 'ржЗрждрзНржпрж╛ржжрж┐', 'ржХрж░рждрзЗ', 'ржЪрзЗрж╖рзНржЯрж╛', 'ржкрзНрж░рждрж┐', 'ржЬржирзЗрж░', 'ржЪрж╛ржи', 'рж╣ржЪрзНржЫрзЗ', 'ржЪрж╛рж░', 'ржирзЗржУрзЯрж╛', 'ржХрж░рзЗржЫрзЗржи', 'ржЫрж┐рж▓рзЗржи', 'ржХрзА',

In [265]:
preprocess("ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА!!!!! рз│")

'ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА рз│'

In [266]:
df['preprocessed_sentence'] = df['sentence'].apply(preprocess) 

In [267]:
df.head()

sentence  hate category  \
0                     ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА!!!!!   1.0   sports   
1                  ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржирзЗржУрзЯрж╛ ржжрж░ржХрж╛рж░   1.0   sports   
2  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ рж╣ржмрзЗ ржПржЯрж╛...   1.0   sports   
3                рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржжрзЗржЦрждрзЗ ржкрж╛ржарж╛рж░ ржоржд ржжрзЗржЦрж╛ ржпрж╛рзЯ   1.0   sports   
4   рждрзБржЗ рждрзЛ рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм   1.0   sports   

                               preprocessed_sentence  
0                          ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА  
1                        ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░  
2  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...  
3                                  рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржкрж╛ржарж╛рж░  
4          рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм

In [268]:
import re

def remove_emojis(text):
    # Define a regex pattern for emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text).strip()

In [269]:
df['preprocessed_sentence'] = df['preprocessed_sentence'].apply(remove_emojis)

In [270]:
df['sentence'][10406]

'ржорж┐рж░рж╛ржЬрзЗрж░ ржЦрзЗрж▓рж╛ ржнрж╛рж▓рзЛ рж▓рж╛ржЧржЫрзЗ ржХрж╛рж░ ржХрж╛рж░ЁЯСЗЁЯСЗ'

In [271]:
df['preprocessed_sentence'][10406]

'ржорж┐рж░рж╛ржЬрзЗрж░ ржЦрзЗрж▓рж╛ ржнрж╛рж▓рзЛ рж▓рж╛ржЧржЫрзЗ ржХрж╛рж░ ржХрж╛рж░'

## Seperate labeled and unlabeled data

In [272]:
df.head(3)

sentence  hate category  \
0                     ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА!!!!!   1.0   sports   
1                  ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржирзЗржУрзЯрж╛ ржжрж░ржХрж╛рж░   1.0   sports   
2  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ рж╣ржмрзЗ ржПржЯрж╛...   1.0   sports   

                               preprocessed_sentence  
0                          ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА  
1                        ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░  
2  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...

In [273]:
df = df.drop(['sentence', 'category'], axis=1)
df.head(3)

hate                              preprocessed_sentence
0   1.0                          ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА
1   1.0                        ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░
2   1.0  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...

In [274]:
# Separate labeled and unlabeled data
labeled_data = df.dropna(subset=['hate'])
unlabeled_data = df[df['hate'].isnull()].drop('hate', axis=1)

In [275]:
labeled_data.head(3)

hate                              preprocessed_sentence
0   1.0                          ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА
1   1.0                        ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░
2   1.0  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...

In [276]:
unlabeled_data.head(3)

preprocessed_sentence
4999  ржорж┐ржирзНржирж┐ ржмрзБржжрж╛рж░ ржмрж┐рждрж░рзЗ ржПрж╕рж┐ржЯ ржорж╛рж░рж╛рж░ ржмрж┐ржЧрж╛рж░ ржЪрзБржжрж╛рж░ ржЖрзЯ ржж...
5000                  ржХрзБрждрзНрждрж╛рж░ ржмрж╛ржЪрзНржЪрж╛рж░рж╛ ржХрзНрж╖ржорждрж╛ ржкрзЗрж▓рзЗ ржкрж╛ржЧрж▓
5001  рж╢рж╛рж▓рж╛рж░ ржмрзЗржЯрж╛рж░рж╛ рж╢рж╛рж▓рзАрж░ ржмрзЗржЯрж┐рж░рж╛ рж╕ржмржХржпрж╝ржЯрж╛ ржорж╛ржжрж╛рж░ржЪрзЛржж ржПржЧрзБ...

## Seperate feature and terget of labeled data

In [277]:
# Split labeled data into features and labels
X_labeled = labeled_data['preprocessed_sentence']
y_labeled = labeled_data['hate']

In [278]:
X_labeled.head(3)

0                            ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА
1                          ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░
2    ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...
Name: preprocessed_sentence, dtype: object

In [279]:
y_labeled.head(3)

0    1.0
1    1.0
2    1.0
Name: hate, dtype: float64

In [280]:
# Split labeled data into training and validation sets
# X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42, stratify=y_labeled)
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, stratify=y_labeled)

In [281]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(7997,) (7997,) (2000,) (2000,)


## Build Model

In [282]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [283]:
# Create a base classifier (e.g., Decision Tree)
base_classifier = DecisionTreeClassifier()

In [284]:
# Create a SelfTrainingClassifier
self_training_classifier = SelfTrainingClassifier(base_classifier)

In [285]:
#1. create a pipeline object
st_clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('st', SelfTrainingClassifier(base_classifier))         
])

In [286]:
# Initial training on labeled data
st_clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('st',
                 SelfTrainingClassifier(base_estimator=DecisionTreeClassifier()))])

In [287]:
# unlabeled_data = unlabeled_data['preprocessed_sentence']
# unlabeled_data

In [288]:
X_train.shape, unlabeled_data['preprocessed_sentence'].shape

((7997,), (10002,))

In [289]:
y_train.shape

(7997,)

In [290]:
X_test

3056                 ржорж╛ржЧрж┐рж░рзЗ ржирж┐рзЯрж╛ ржЖрж╕ ржирж┐ржЪ рждрж╛рж▓рж╛рзЯ ржХрж▓рж╛ ржЧрж╛ржЫ рждрж┐ржорзБ
2280     ржмрж╛ржВрж▓рж╛ржжрзЗрж╢рзЗрж░ ржкрж░рж┐ржЪрж╛рж▓ржХ ржЧрзБрж▓рзЛ ржиржЯрж┐рж░ ржЫрзЗрж▓рзЗредржЖрж░ ржирж╛рзЯрж┐ржХрж╛ ржЧрзБ...
12987                                рж╣рзГржжрзЯ ржирж╛рж░рж╛ ржЫржмрж┐ ржЕрж╕рж╛ржзрж╛рж░ржг
13787                         ржЯрж╛ржХрж╛рж░ рж╢рж╛ржмржирзВрж░ ржирж┐ржорзНржи ржорж╛ржирзЗрж░ ржЫржмрж┐
14958    рж╕рж┐ржирзЗржорж╛ржЯрж╛рзЯ ржЫрж┐ржи ржЧрж╛ржи ржХрж╛ржЯрж╛редржкрзБрж░рж╛ржкрзБрж░рж┐ ржжрж┐рж▓рзЗ ржнрж╛рж▓рзЛ рж╣рждрзЛред...
                               ...                        
4530              ржорж╛ржЧрзА рж╕рзНржпрж╛рж░рзЗрж░ ржжрзЗрж╛рж╖ ржирж╛ржЗредржЖржХрж╛ржо ржжрзЗрж╛рж╖ ржЯрж╛ ржХрж┐рж╕рзЗрж░
843                                               ржмрж▓ржжрзЗрж░ ржжрж▓
10242                             ржЖржЪрзНржЫрж╛ ржПржирж╛ржорзБрж▓рзЗрж░ рж╣рж╛ржЗ рж╕рзНржХрзЛрж░
10540    рж░рзБржмрзЗрж▓ рж╕ржмрж╛ржЗ ржЧ

In [291]:
y_test[:15]

3056     1.0
2280     1.0
12987    0.0
13787    0.0
14958    0.0
13750    0.0
358      1.0
686      1.0
1741     1.0
11051    0.0
13729    0.0
652      1.0
1091     1.0
3543     1.0
4799     1.0
Name: hate, dtype: float64

In [292]:
# Pseudo-labeling and iterative training
for iteration in range(15):
    # Pseudo-labeling on unlabeled data
    pseudo_labels = st_clf.predict(unlabeled_data['preprocessed_sentence'])

    # Combine labeled and pseudo-labeled data
    X_combined = pd.concat([X_train, unlabeled_data['preprocessed_sentence']], axis=0)
    y_combined = pd.concat([y_train, pd.Series(pseudo_labels)], axis=0)

    # Train the self-training classifier on the combined dataset
    st_clf.fit(X_combined, y_combined)
    
    y_pred = st_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.646


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.652


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


...

0.649


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.6545


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.6525


In [293]:
len(X_combined), len(y_combined)

(17999, 17999)

In [294]:
X_combined

14662                             ржПрж░ржХржо ржирж╛рзЯржХ рж╢рж╛ржмржирзВрж░ ржорж╛ржирж╛ржирж╕ржЗ
11928    ржЖрж╕рж▓ ржмрж▓рзЗржирж╛ ржХрж╛рж░рзЗрж╛ рж╕рж╛рж╣рж╕ ржирж╛ржЗржорзБрж╕рж▓рж┐ржо ржжрзЗрж╢рзЗ рж╕рж░ржХрж╛рж░ рж░рж╛рж╖рзН...
10341                           ржЬрж╛рж░рж╛ ржкрж╛ржкржи ржЦрж╛рж░рж╛ржк ржнрзБрж▓ ржнрж╛ржмржЫрзЗржи
10672    ржорзЛржкрж╛рж╕рж░ ржорж┐рждрж╛ ржХрждрж╛ ржжрж░рж╛ ржЧрзЗржЪ ржХрж░рж╛ржи ржЧрзБрж╕рзЗрж░ ржЯрж╛ржХрж╛ ржмрж╛ржЧ ржкрж╛...
12909    ржЫрж┐ ржПржЧрзБрж▓рзЛ ржЫрзЗрж╣рж╛рж░рж╛ ржирж╛ржЗржвржХ рждрзЛржШрж░рзЗ ржкрж╛ржЗрж▓рзЗ ржШржирзНржЯ ржЬрзБрждрж╛ ржмрж╛...
                               ...                        
19994    рж▓рзБржЪрзНржЫрж╛ рж╣рзБржЬрзБрж░ржЧрзБрж▓рзЛржХрзЗ ржорзБржХрж┐ ржкрж╛ржЪрж╛ржЗ рж▓рж╛рждрж┐ ржорзЗрж░рзЗ ржжрзЗрж╢ рждрж╛...
19995                                   ржХрзБрждрзНрждрж╛ржЯрж╛ ржлрж╛ржБрж╕рзА ржжрж╛ржУ
19996          ржЧрж┐рзЯрж╛рж╕ржЙржжрзНржжрж┐ржи ржПрж▓рж╛ржХрж╛рзЯ ржЦржмрж░ ржПржирзЗ ржорж▓ржо рж▓рж╛

In [295]:
y_combined.shape

(17999,)

In [296]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, stratify=y_combined)

In [297]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('MNB', MultinomialNB())         
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


#4. print the classfication report
print(classification_report(y_test, y_pred))

0.7333333333333333
              precision    recall  f1-score   support

         0.0       0.76      0.58      0.66      1589
         1.0       0.72      0.85      0.78      2011

    accuracy                           0.73      3600
   macro avg       0.74      0.72      0.72      3600
weighted avg       0.74      0.73      0.73      3600



In [298]:
df

hate                              preprocessed_sentence
0       1.0                          ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА
1       1.0                        ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░
2       1.0  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...
3       1.0                                  рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржкрж╛ржарж╛рж░
4       1.0          рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм
...     ...                                                ...
19994   NaN  рж▓рзБржЪрзНржЫрж╛ рж╣рзБржЬрзБрж░ржЧрзБрж▓рзЛржХрзЗ ржорзБржХрж┐ ржкрж╛ржЪрж╛ржЗ рж▓рж╛рждрж┐ ржорзЗрж░рзЗ ржжрзЗрж╢ рждрж╛...
19995   NaN                                 ржХрзБрждрзНрждрж╛ржЯрж╛ ржлрж╛ржБрж╕рзА ржжрж╛ржУ
19996   NaN        ржЧрж┐рзЯрж╛рж╕ржЙржжрзНржжрж┐ржи ржПрж▓рж╛ржХрж╛рзЯ ржЦржмрж░ ржПржирзЗ ржорж▓ржо рж▓рж╛ржЧрж╛рзЯрзЗ рж▓рж╛ржЧржмрзЛ
19997   NaN                             рж╣рзБржЬрзБрж░ ржкрж┐ржЯрж┐рзЯрзЗ ржмрзЗрж░ ржжрж░ржХрж╛рж░
19998   NaN  ржорж╣рж┐рж▓рж╛рж░ ржорж╛ржерж╛рж░ ржЪрзБрж▓ ржмрзИрж╢рж╛ржЦ ржорж╛рж╕рзЗрж░ ржЦрзЗрж░рзЗрж░ ржкрж╛рж▓рж╛ ржЧрж░рзБржХрзЗ ...

[19999 rows x 2 columns]

In [299]:
import re

def remove_emojis(text):
    # Define a regex pattern for emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['sen'] = df['preprocessed_sentence'].apply(remove_emojis)
df

hate                              preprocessed_sentence  \
0       1.0                          ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА   
1       1.0                        ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░   
2       1.0  ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...   
3       1.0                                  рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржкрж╛ржарж╛рж░   
4       1.0          рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм   
...     ...                                                ...   
19994   NaN  рж▓рзБржЪрзНржЫрж╛ рж╣рзБржЬрзБрж░ржЧрзБрж▓рзЛржХрзЗ ржорзБржХрж┐ ржкрж╛ржЪрж╛ржЗ рж▓рж╛рждрж┐ ржорзЗрж░рзЗ ржжрзЗрж╢ рждрж╛...   
19995   NaN                                 ржХрзБрждрзНрждрж╛ржЯрж╛ ржлрж╛ржБрж╕рзА ржжрж╛ржУ   
19996   NaN        ржЧрж┐рзЯрж╛рж╕ржЙржжрзНржжрж┐ржи ржПрж▓рж╛ржХрж╛рзЯ ржЦржмрж░ ржПржирзЗ ржорж▓ржо рж▓рж╛ржЧрж╛рзЯрзЗ рж▓рж╛ржЧржмрзЛ   
19997   NaN                             рж╣рзБржЬрзБрж░ ржкрж┐ржЯрж┐рзЯрзЗ ржмрзЗрж░ ржжрж░ржХрж╛рж░   
19998   NaN  ржорж╣рж┐рж▓рж╛рж░ ржорж╛ржерж╛рж░ ржЪрзБрж▓ ржмрзИрж╢рж╛ржЦ ржорж╛рж╕рзЗрж░ ржЦрзЗрж░рзЗрж░ ржкрж╛рж▓рж╛ ржЧрж░рзБржХрзЗ ...   

                                                     sen  
0                              ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА  
1                            ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░  
2      ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржк...  
3                                      рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржкрж╛ржарж╛рж░  
4              рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм  
...                                                  ...  
19994  рж▓рзБржЪрзНржЫрж╛ рж╣рзБржЬрзБрж░ржЧрзБрж▓рзЛржХрзЗ ржорзБржХрж┐ ржкрж╛ржЪрж╛ржЗ рж▓рж╛рждрж┐ ржорзЗрж░рзЗ ржжрзЗрж╢ рждрж╛...  
19995                                 ржХрзБрждрзНрждрж╛ржЯрж╛ ржлрж╛ржБрж╕рзА ржжрж╛ржУ  
19996        ржЧрж┐рзЯрж╛рж╕ржЙржжрзНржжрж┐ржи ржПрж▓рж╛ржХрж╛рзЯ ржЦржмрж░ ржПржирзЗ ржорж▓ржо рж▓рж╛ржЧрж╛рзЯрзЗ рж▓рж╛ржЧржмрзЛ  
19997                             рж╣рзБржЬрзБрж░ ржкрж┐ржЯрж┐рзЯрзЗ ржмрзЗрж░ ржжрж░ржХрж╛рж░  
19998  ржорж╣рж┐рж▓рж╛рж░ ржорж╛ржерж╛рж░ ржЪрзБрж▓ ржмрзИрж╢рж╛ржЦ ржорж╛рж╕рзЗрж░ ржЦрзЗрж░рзЗрж░ ржкрж╛рж▓рж╛ ржЧрж░рзБржХрзЗ ...  

[19999 rows x 3 columns]

In [302]:
df['preprocessed_sentence'][10540]

'рж░рзБржмрзЗрж▓ рж╕ржмрж╛ржЗ ржЧрж╛рж▓рж╛ржЧрж╛рж▓рж┐ ржжрж┐рждрзЗржЫрзЗ < br />,,,ржЖрж░ ржирж╛рж╕рж┐рж░ ржХрж░рждрзЗржЫрзЗ ржкрзНрж░ржорж╛ржи рж╕рзБржмрж╛ ржмрж▓рждрзЗржЫрзЗ ржХрзЗржУ ржорж╛ржирждрзЗ ржирж╛рж░рж╛ржЬ < br />,,,рж╕ржмрж╛рж░ рж╕рзБржмрж╛ ржЦрж░рж╛ржк ржирж╛рж╕рж┐рж░рзЗрж░ ржжрзЛрж╖ ржЖржкржирж╛рж░рж╛ ржирж╛рж╕рж┐рж░ рж╕рзБржмрж╛рж░ ржЧрзБрж▓рж╛ ржХрж▓ рж░рзЗрж░рзНржХржб ржЧрзБрж▓рж╛ рж╢рзБржирзБржи ржмрж▓рзБржи ржЦрж╛рж▓рж┐ рж░рзБржмрзЗрж▓ ржирж╛рж╕рж┐рж░ рж╕рж╛ржмрзНржмрж┐рж░ ржХржерж╛ржУ ржмрж▓рзБржи ржХрзЗржирзЛ ржЧрж╛рж▓рж┐ ржжрж┐ржмрзЗржи'

In [301]:
df['sen'][10406].strip()

'ржорж┐рж░рж╛ржЬрзЗрж░ ржЦрзЗрж▓рж╛ ржнрж╛рж▓рзЛ рж▓рж╛ржЧржЫрзЗ ржХрж╛рж░ ржХрж╛рж░'

In [306]:
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.text)
    
    return "/".join(filtered_tokens) 

In [307]:
preprocess('рж░рзБржмрзЗрж▓ рж╕ржмрж╛ржЗ ржЧрж╛рж▓рж╛ржЧрж╛рж▓рж┐ ржжрж┐рждрзЗржЫрзЗ < br />,,,ржЖрж░ ржирж╛рж╕рж┐рж░ ржХрж░рждрзЗржЫрзЗ ржкрзНрж░ржорж╛ржи рж╕рзБржмрж╛ ржмрж▓рждрзЗржЫрзЗ ржХрзЗржУ ржорж╛ржирждрзЗ ржирж╛рж░рж╛ржЬ < br />,,,рж╕ржмрж╛рж░ рж╕рзБржмрж╛ ржЦрж░рж╛ржк ржирж╛рж╕рж┐рж░рзЗрж░ ржжрзЛрж╖ ржЖржкржирж╛рж░рж╛ ржирж╛рж╕рж┐рж░ рж╕рзБржмрж╛рж░ ржЧрзБрж▓рж╛ ржХрж▓ рж░рзЗрж░рзНржХржб ржЧрзБрж▓рж╛ рж╢рзБржирзБржи ржмрж▓рзБржи ржЦрж╛рж▓рж┐ рж░рзБржмрзЗрж▓ ржирж╛рж╕рж┐рж░ рж╕рж╛ржмрзНржмрж┐рж░ ржХржерж╛ржУ ржмрж▓рзБржи ржХрзЗржирзЛ ржЧрж╛рж▓рж┐ ржжрж┐ржмрзЗржи')

'рж░рзБржмрзЗрж▓/рж╕ржмрж╛ржЗ/ржЧрж╛рж▓рж╛ржЧрж╛рж▓рж┐/ржжрж┐рждрзЗржЫрзЗ/</br//>,,,ржЖрж░/ржирж╛рж╕рж┐рж░/ржХрж░рждрзЗржЫрзЗ/ржкрзНрж░ржорж╛ржи/рж╕рзБржмрж╛/ржмрж▓рждрзЗржЫрзЗ/ржХрзЗржУ/ржорж╛ржирждрзЗ/ржирж╛рж░рж╛ржЬ/</br//>,,,рж╕ржмрж╛рж░/рж╕рзБржмрж╛/ржЦрж░рж╛ржк/ржирж╛рж╕рж┐рж░рзЗрж░/ржжрзЛрж╖/ржЖржкржирж╛рж░рж╛/ржирж╛рж╕рж┐рж░/рж╕рзБржмрж╛рж░/ржЧрзБрж▓рж╛/ржХрж▓/рж░рзЗрж░рзНржХржб/ржЧрзБрж▓рж╛/рж╢рзБржирзБржи/ржмрж▓рзБржи/ржЦрж╛рж▓рж┐/рж░рзБржмрзЗрж▓/ржирж╛рж╕рж┐рж░/рж╕рж╛ржмрзНржмрж┐рж░/ржХржерж╛ржУ/ржмрж▓рзБржи/ржХрзЗржирзЛ/ржЧрж╛рж▓рж┐/ржжрж┐ржмрзЗржи'

In [313]:
df['preprocessed_sentence'][15]

'ржмрзНржм'

In [312]:
for i in df['preprocessed_sentence']:
    print(i)

ржпрждрзНрждрж╕ржм ржкрж╛ржкржи рж╢рж╛рж▓рж╛рж░ ржлрж╛ржЬрж▓рж╛ржорзА
ржкрж╛ржкржи рж╢рж╛рж▓рж╛ рж░рзЗ рж░рж┐ржорж╛ржирзНржбрзЗ ржжрж░ржХрж╛рж░
ржЬрж┐рж▓рзНрж▓рзБрж░ рж░рж╣ржорж╛ржи рж╕рзНржпрж╛рж░рзЗрж░ ржЫрзЗрж▓рзЗ ржПрждрзЛ ржмрзЬ ржЬрж╛рж░ржЬ ржнрж╛ржмрждрзЗ ржкрж╛рж░рзЗржирж┐редржзржирзНржпржмрж╛ржж рждрж╛рж╣рж╕рж┐ржи ржнрж╛ржЗ
рж╢рж╛рж▓рж╛ рж▓рзБржЪрзНржЪрж╛ ржкрж╛ржарж╛рж░
рж╢рж╛рж▓рж╛ ржЧрж╛ржЬрж╛ ржЦрж╛ржЗржЫржЪредрждрзБрж░ ржорж╛рж░ рж╣рзЗржбрж╛рзЯ ржЦрзЗрж▓ржмрзЗ рж╕рж╛ржХрж┐ржм
ржХрзБржи ржжрж░ржирзЗрж░ ржХрзЗрж▓рж╛ ржлрж╛ржЗржЬрж▓рж╛ржорж┐ рждрж╛рж╕рзНржХрж┐ржи рж░рзЗ ржЪрж░ ржорж╛рж░рж╛ ржжрж░ржХрж╛рж░
ржкрж╛ржкржи ржнрж░ ржорж╛ржжрж╛ ржЪрзЛржж ржкрж╛ржкржирзЗ ржкржжрждрзЗржХ ржЪрж╛ржЗ
ржжрзБрж░рзЛ рж╕рж╛рж▓рж╛рж░ ржкрзБржж ржЪрзБржкржерж╛ржХ
ржХрзБрждрзНрждрж╛рж░ ржмрж╛ржЫржЪрж╛ ржкрж╛ржкржи
ржмрж╛рж▓ ржЫрж╛рж▓ рждрж░ рж╕рж╛ржЙрзЯрж╛
ржХржкрж╛рж▓рзЗ ржЬрзБрждрж╛ ржорж╛рж░рж┐ рж╢рж╛рж▓рж╛рж░ ржкрзБржд
ржкрж╛ржкржирзЗ ржкрж╛ржЧрж▓ рж╣рзЯрж╛ржЫрзЗ
ржжрзЗржЦрзЗржЫрзЛ ржорж╛р

ржорж╛ржжрж╛рж░ ржЪрзЛржж ржЫрзЗржХржЪрзЗрж░ ржнрж┐рзЬрж┐ржУ 23 ржорж┐ржгрж┐ржЯ ржлрж╛рж▓ржЯрзЛ ржЧрзЗрж▓рзЛ
ржЪрзЗрж▓рзЗржи ржжрзЗржЦ рж╕рж╛рж▓рж╛рж░ ржмрж╛ржЪрзНржЫрж╛ рж░рж╛ ржЕржкржорж╛ржи ржЧрзНрж░рж┐ржирж╛ рж▓рж╛ржЧрзЗ ржХрзЯржХржЯрж╛ ржЬрж╛ржирзЛрзЯрж╛рж░ ржорж╛рж╕рзБржж рж░рж╛ржирж╛ржХрзЗ ржЦрзБржЬржЫрж┐ржЫ рж▓рж╛ржн ржЬрзАржмржирзЗржУ ржорж╛рж╕рзБржж рж░рж╛ржирж╛ ржкрж╛ржмрж┐ ржХрзБрждрзНрждрж╛ ржорж┐ржбрж┐рзЯрж╛
рж░рж╛ржЬ ржнрж╛ржЗ рж╢рж┐ржЦ ржмрж╛ржХрзАрж░рж╛ ржпрждрж╕ржм ржЖржмрж╛рж▓рзЗрж░ ржжрж▓редрж░рж╛ржЬ ржнрж╛ржЗ рж╕рзЗрж░рж╛ ржжрж╛рзЬрж┐ ржУрзЯрж╛рж▓рж╛ ржЪрж╛ржирзНржбрж╛рж▓ рж╢рж┐ржЦрж╛рж░
ржХржорзЗржирзНржЯ ржкрзЬрждрзЗ ржЖрж╕рж╛ ржирзЗржХрзНрж╕ржЯ ржПржкрж┐рж╕рзЛржб ржЕржВрж╢ ржпрж╛ржУрзЯрж╛рж░ ржХрзЯрзЗржХржЯрж╛ ржЫрзЗржБрзЬрж╛ ржЬрзБрждрж╛ ржлржЗржирзНржирж┐ ржорж╛рж░рзНржХрж╛ ржЖржмрж╛рж▓ржжрзЗрж░ ржорзБржЦрзЗ ржорзЗрж░рзЗ ржЖрж╕рж╛рж░ ржЕржирзБрж░рзБржз рж░ржЗрж▓рзЛ
рждрж╛ржиржнрзАрж░ ржорж╛ржпрж╝рзЗрж░ ржХржмрж░ ржЬрж┐ржпрж╝рж╛рж░ржд ржорж╣рж┐рж▓рж╛рж░ ржЫржмр

рж╢рж╛рж▓рж╛ ржорзБрж░ржЧрзА ржХржмрж┐рж░
ржЦрж╛ржиржХрж┐рж░ ржкрзБрждрзЗрж░рж╛рж░рзЗ ржЬрзБрждрж╛ ржжрж┐рзЯрж╛ ржкрж┐ржЯрж╛
ржнржирзНржб ржжрзЗржУрзЯрж╛ржи ржмржЧрж┐ржХ ржХрзБрждрзНрждрж╛рж░ржмрж╛ржЪрзНржЪрж╛ржХрзЗ ржЬрзБрждрж╛рж░ ржмрж╛рж░рж┐
рж╢рж╛рж▓рж╛рж░ ржмрзЗржЯрж╛ ржнрж╛рж╖рж╛ ржХрж░редржмрзНржпржмрж╣рж╛рж░ рж╢рзЗржЦ ржмрзЗржпрж╝рж╛ржжржм
ржЬрж╛ржирзЛржЖрж░рзЗрж░ ржмрж╛ржЪрзНржЪрж╛ ржорж░ржмрж┐ржирж╛
рж╢рж╛рж▓рж┐ржХрзЗ ржЬрзБрждрж╛ ржкрж┐ржЯрж╛ржмрзЛ
ржХрзБрзНрждрзНрждрж╛рж░ ржмрж╛ржЪрзНржЪрж╛ржжрзЗрж░ ржЬрзБрждрж╛ ржорж╛рж░рзЛ
ржЦрж╛ржиржХрж┐рж░ ржкрзЛрж▓рж╛ рждрж░рзЗ ржкрж╛рзЯрж▓рзЗ ржкрзБржЯрж┐ржХрж┐ ржЖрж╕рзНржд ржмрж╛рж╢ ржбрзБржХрж╛рждрж╛ржо
ржХрзБрж▓рж╛ржЩрзНржЧрж╛рж░ рж╕ржирзНрждрж╛ржи
рж╢рж╛рж▓рж╛рж░рзЗ рж╕ржмрж╛ржЗ ржорж┐рж▓рзНрж▓рж╛ ржЬрзБрждрзНрждрж╛ржЯрж╛ ржжрж┐рзЯрж╛ ржкрж┐ржЯрж╛рзЯ
ржлрж╛ржЯржлрж╛рж░рзЗрж░ ржмрж╛ржЪрзНржЪрж╛ ржЕржирзНржжрзБ ржЪрзНржпрж╛ржирзЗрж▓ ржЦрзБрж▓рж┐рзЯрж╛ ржлрж╛ржЬрж▓рж╛ржорж┐ ржХрж░ржЫ
рж╢рж╛рж▓рж╛ ржХрзБрждрзНрждрж╛рж░ ржмрж╛ржЪрзНржЪ

ржиржбрж┐рж░ржкрзБржд   ржХржорзЗржирзНржЯрзЗ ржЖрзЯ рж░рзЗржгрзНржбрзАржорж╛ржЧрж┐рж░ржкрзБржд
рж╖рж╛ржбрж╝ ржирж╛рж╕рзНрждрж┐ржХ ржжрж╛рж▓рж╛рж▓ рж╕рж░рж┐ ржХрзЛржжрж╛рж▓ ржЗржХржмрж╛рж▓ ржПрждржжрж┐ржи ржХрзЛржерж╛ржпрж╝
ржмрж╛ржЬрзЗ ржзрж░ржирзЗрж░ рж▓рзБржЗржЪрзНржЪрж╛ ржШрж░рзЗрж░ ржШрж╛ржЯрзЗрж░ рж╣рж┐ржЬрж░рж╛ ржЯрж╛ржЗржк
ржорж┐рзЯрж╛ ржЖржмрж╛рж▓ржЪрзЛржжрж╛ ржХржУ
ржЬрж╛ржлрж░ ржЗржХржмрж╛рж▓ рж╕рзНржпрж╛рж░рзЗрж░ ржЖржорж┐ржУ ржорзБржЦрзЗ ржЖржЩрзНржЧрзБрж▓ ржмржорж┐ ржХрж░ржм рж╕рзЗрж╕ржм ржХрзБрж▓рж╛ржЩрзНржЧрж╛рж░ржжрзЗрж░ ржЧрж╛рзЯрзЗ рж░рж╛ржЬрж╛ржХрж╛рж░ рж▓рж┐ржЦрзЗ рж░рж╛ржЬржкржерзЗ ржжрж╛ржБрзЬрж┐рзЯрзЗржЫрж┐рж▓
ржкрж╛ржЧрж▓ ржЪрзБржжрждрзЗ ржЖржЗрж▓рж╛ржо
ржЪрзЛржжржирж╛ ржХрзЛржерж╛ржХрж╛рж░ ржирж┐ржЬрзЗржХрзЗ ржмрзЬ ржмрзЛржХрж╛ржЪрзЛржжрж╛
ржХрждржЧрзБрж▓рзЛ ржЖржмрж╛рж▓рзЗрж░ ржХржорзЗржирзНржЯрзЗрж╕ ржкрзЬрж▓рзЗ ржЬржирзНржорзЗ ржкрж╛ржХрж┐ржжрзЗрж░ ржмрж┐рж░рзНржЬ ржмрзНржпржмрж╣рзГржд рж╣ржЗржЫрзЗ ржмрзЗржЬржирзНржорж╛ржЧрзБрж▓ ржкрзЬрзЗ ржмрж╛ржЬрзЗ ржХрж

ржлржХрж┐рж░рзЗрж░ ржЯрж╛ржХрж╛ ржЖрж░ржХрж┐
ржкрж╛ржкржи ржкржжрждрзНржпрж╛ржЧ ржЪрж╛ржЗ
рж╣ржЗржЫрзЗрждрзЛ ржЫрж╛ржЗрзЬрж╛ ржнрж╛ржЗрж░рж╛рж▓ ржжрж░ржХрж╛рж░
ржврж╛ржХрж╛рж░ ржнржХрзНрждрж░рж╛ ржХржорзЗржирзНржЯ ржХрж░рзБржи ржЦрзБрж▓ржирж╛рж░ ржнржХрзНрждрж░рж╛ рж▓рж╛ржЗржХ
рж╕рзМржорзНржп рждрж╛рж╕ржХрж┐ржиржХрзЗ ржжрж▓рзЗрж░ рж╕рж┐рж▓рзЗржХрзНржЯрж░ ржорзНржпрж╛ржирзЗржЬрж╛рж░ ржмрж╛ржирж┐рзЯрзЗ ржжрж┐рж▓рзЗржЗ рж▓рзНржпрж╛ржарж╛ ржЪрзБржХрзЗ рж╕рзМржорзНржп рждрж╛рж╕ржХрж┐ржиржХрзЗ ржЪрж╛ржирзНрж╕ ржжрж▓рзЗ ржПржХрзЗржмрж╛рж░рзЗ ржкрж╛ржХрж╛ рж╕рзНржерж╛ржи
ржзржирзНржпржмрж╛ржж ржнрж╛ржЗржпрж╝рж╛ ржЖржкржирж╛ржХрзЗ рж╕рзБржирзНржжрж░ ржирзИрждрж┐ржХ ржмрж╛ржЪржХ ржмрж▓рж╛рж░ рж╣рзБржо рж╕рж╛ржХрж┐ржм ржнрж╛ржЗржпрж╝рзЗрж░ ржкрж╛рж╢рзЗ ржЖржЫрж┐ ржерж╛ржХржмрзЛ ржЗржирж╢рж╛ржЖрж▓рзНрж▓рж╛рж╣
ржЦрзЗрж▓рждрзЗ ржЦрж╛рж▓рж┐ ржЯрж╛ржХрж╛ ржЯрж╛ржХрж╛ ржХрж░рж╛рзЗ рж╕рж╛рж▓рж╛рж░рж╛ ржЦрж╛рж▓рж┐ ржЯрж╛ржХрж╛рж░ рж▓рзБржмрж┐
ржмрж╛ржВрж▓рж╛ржжрзЗрж╢ ржЯрж┐ржХрзЗржЯ ржжржВржЦрж╖рзЛ ржжрж

ржПржбрж╛ ржХрзНржпрж╛ржоржи рж╢рзЗрж╖рзЗ рж╕ржмрж╛ржЗ рж╢рж╛ржмржирзБрж░рзЗрж░ ржорж╛ржл ржЪрж╛ржЗрж▓рзЛ
ржирж╛ржЗржЪ ржирж╛ржЯрзЛржХ,,0096597621520/
ржПржбржорж┐ржи ржнрж╛ржЗ ржЬрж╛ржмрзЗржж ржирж┐рж╢рж╛ржи ржорзБржнрж┐ржЯрж╛ ржЖржкрж▓рзЛржб ржжрзЗржмрзЗржи ржХрзЗржоржи
ржорж╛рж░ржЬрзБржХ рж░рж╛рж╕рзЗрж▓ ржЕржнрж┐ржирзЗрждрж╛ рж╕рж┐ржжрзНржжрж┐ржХ рж╣ржЗржд
ржкрзБрж░рзЛржЯрж╛ ржЫржмрж┐ рж╕рзБржирзНржжрж░ рж▓рзЛржХрзЗрж╢ржирзЗ рж╕рж┐ржирзЗржорж╛ ржмрж╛ржВрж▓рж╛рж░ ржЗрждрж┐рж╣рж╛рж╕рзЗ ржирж┐
ржХрж╖рзНржЯ ржкрзЗрж▓рж╛ржо,,ржХрж╛ржирзНржирж╛ ржПрж╕рзЗржЫрзЗ
ржкрзНрж░рждрж┐ржЯрж╛ рж░рзЛржЧ ржПрж░ржХржо ржирж╛ржЯржХ ржмрж╛ржирж╛ржирзЛрж░ ржЕржирзБрж░рзЛржз ржХрж░ржЫрж┐
рж╢рж╛ржмржирзВрж░ рж░рзБржмрзЗрж▓рзЗрж░ ржорзБржнрж┐ ржЯрж╛ ржЦрзБржмржЗ ржнрж╛рж▓рзЛ рж▓рж╛ржЧржЫрзЗ
ржЧрж▓рзНржк рж╕рзБржирзНржжрз▒
ржПржХрзЗрждрзЛ ржжрзЗрж░ ржЦрж╛рж░рж╛ржк ржжрзНржмрж┐рждрзАрзЯрждрзЗ ржзрзЛржХрж╛ржмрж╛ржЬрж┐ ржЪрзНржпрж╛ржирзЗрж▓ ржЖржЗрзЯрзЗрж░ ржЙржкржпрзБржХрзНржд ржмрж┐ржЪрж╛рж░ ржЪрж╛ржЗ
ржжрзЗрж

ржжрж╛рзЬрж┐ржУрзЯрж╛рж▓рж╛ рж╢рзЯрждрж╛ржирж╛ ржХрж╛ржорж╛рж▓ ржорзБрж░ржЧрж┐ ржХржмрж┐рж░ ржЯржХрж╢рзЛ ржХрж░рждрзЛ
рж░рж╛рзЯ ржЪрзЯрзЗржЫрж┐рж▓ ржмрж╛ржВрж▓рж╛рж░ ржЬржиржЧржг ржПржЗржнрж╛ржмрзЗ ржкрждрж┐ржЯрж┐ ржмрж┐ржЪрж╛рж░ ржжрж░ржХрж╛рж░
ржЦрж╛рж░рж╛ржк
ржорж┐ржирзНржирж┐ рж░рж┐ржлрж╛ржд ржнрж╛ржЗрж░рж╛рж▓ ржЫрж┐рж▓рзЛ ржкрзНрж░рж┐рзЯрж╛ рж╕рж╛рж╣рж╛ ржЬрж╛рзЯржЧрж╛ ржирж┐рж▓рзЛ ржкрзНрж░рж┐рзЯрж╛ рж╕рж╛рж╣рж╛ ржнрзБрж▓рзЗ ржЧрзЗрж▓рзЛ ржХрж▓рзНрж▓рж╛ ржХрж╛ржЯрж╛рж░ ржЙржжрзНржнржм ржХрж▓рзНрж▓рж╛ржХрж╛ржЯрж╛ ржерзЗржорзЗ ржЧрзЗрж▓рзЛ ржбрзЗржВржЧрзБ ржорж╢рж╛ ржЬржирзНржо ржирж┐рж▓рзЛ ржбрзЗржВржЧрзБ ржорж╢рж╛рж░ рж╢рзЗрж╖ ржЪрж╛ ржврж╛рж▓рж╛рж░ ржмрж╛ржВрж▓рж╛ржжрзЗрж╢рзЗ рж░рзБржорзЗ ржЫрж┐рж▓рзЛ ржПрж╕рж┐ ржЖрзЬрж╛рж▓рзЗ ржЫрж┐рж▓рзЛ рж╕рж┐рж╕рж┐ ржЖрж╣рж╛рж░рзЗ ржмрзЗржЪрж╛рж░рж╛ ржбрж┐рж╕рж┐ ржжрзЗрж╢ржмрж╛рж╕рзА ржжрж┐ржЪрзНржЫрзЗ ржЫрж┐ржГ ржЫрж┐ржГ
рж╕ржмрж╛ржЗ ржЫрж┐ ржЫрж┐ ржжрзЛрж╖ ржПржХрж╛рж░ ржбрж┐ рж╕рж┐
ржжрзЛрж╖ ржжрзЗрж╢рзЗ ржкрж░рж┐ржорж╛ржи рж░рзЗржк ржкрж